### Load Materials Project API Key

In [1]:
import os, stat
from IPython.display import clear_output

try:
    user = str(input())
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user == None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

Success


## Libraries

This notebook requires several libraries to be imported. They are separated in blocks depending on their usage.

In [94]:
# These lines import both libraries and then define an array with elements to be used below

#Pymatgen and sub-libraries
import pymatgen
from pymatgen import MPRester, Element, Structure
from pymatgen.core import composition
from pymatgen.io.vasp import Vasprun
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
from pymatgen.util.plotting import *
from pymatgen.analysis.phase_diagram import *
#import pymatgen.core.composition
from pymatgen.apps.borg.hive import VaspToComputedEntryDrone
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
import json

#Misc. Tools
import matplotlib.pyplot as plt
import random
import os
from itertools import combinations
import pandas as pd

#Import API Key
file = open(os.path.expanduser('~/.mpkey.txt'),"r+")
apikey = file.readline()
file.close()
rester = MPRester(apikey)

## 1. Query all selected materials  in MP database

In [260]:
#Query every structure in the MP database that has oxygen atom within compound. 
#Grab set of properties from database. Ti, Fe, Ni, W, C, Mo based.
# More elements "Si", "Zr", "V", "N", "B", "Ta", "Nb", "Hf", "Cr"
# We will have this setup as a data pipeline extraction procedure as iterations.
data = rester.query({"elements": "Si",
                     "nelements": {"$gte": 2}},
                    ["task_id","pretty_formula","formula","volume","density","elements",
                     "e_above_hull","elasticity","unit_cell_formula"])

In [261]:
print('Number of oxide structures available on the MP database: %s' % len(data))
print('Example output: %s' % data[77])

Number of oxide structures available on the MP database: 8932
Example output: {'task_id': 'mp-1073958', 'pretty_formula': 'Mg6Si5', 'formula': {'Mg': 6.0, 'Si': 5.0}, 'volume': 427.4584954039655, 'density': 2.224036997059014, 'elements': ['Mg', 'Si'], 'e_above_hull': 0.2136582768181814, 'elasticity': None, 'unit_cell_formula': {'Mg': 12.0, 'Si': 10.0}}


## 2. Filtering data for energy stability
After data query has been completed we will begin filtering things just a bit. First we will strip any values that have an energy above the convex hull greater than 1 meV. 0 meV indicates most stable phase. This filter can be changed below.<br>


In [262]:
energy_cutoff_value = 1 #energy above convex hull cutoff [meV]

### 2.1 Filter data and calculate ionic packing fraction from compound query

In [266]:
#Create temporary list arrays for data calculation and manipulation. We will clear the array after each loop
atom_sample = []
atom_count = []
atom_radii = []
atom_volume = []
atom_volume_total = []

#Create modified dictionary to store values that we want from queried database of all oxides
property_dict_all = {}

for k in range(0,len(data)):
    #If Energy above convex Hull is larger than 1 meV we will skip that value and move on
    e_above_hull = data[k].get('e_above_hull')*1000
    print(e_above_hull)
    if e_above_hull > energy_cutoff_value:
        continue
        
    #Grab values from data query. Data stored as dictionary so we call the 'keys' for 'values'
    mp_id = data[k].get('task_id')
    density = data[k].get('density')
    pretty_formula = data[k].get('pretty_formula')
    elastic_tensor = data[k].get('elasticity')
    composition = data[k].get('formula')
    oxide_volume = data[k].get('volume')
    atom_type_count = len(data[k].get('elements'))
    unit_cell_formula = data[k].get('unit_cell_formula')

    #Pull atom count for each atom type. Will be used to find summed atomic volumes and summed masses
    for type in np.arange(0,atom_type_count):
        atom_sample.append(data[k].get('elements')[type])     
        atom_count.append(data[k].get('unit_cell_formula').get(data[k].get('elements')[type]))
        
    #Do a quick query of each element type for ionic radius and atomic mass from Pymatgen
    for item in atom_sample:
        element_object = pymatgen.Element(item)
        radii_values = element_object.average_ionic_radius
        atom_radii.append(radii_values)

    #For each atom radii queried calculate the atomic volume
    for i in atom_radii:
        v = (4/3)*np.pi*i**3
        atom_volume.append(v)
        
    #Calculate the total atomic volume for each element type (E.g. 3*(Volume O atom))
    for i in np.arange(0,len(atom_count)):
        v_total = atom_count[i]*atom_volume[i]
        atom_volume_total.append(v_total)
        
    #Sum the volumes of all atoms in the oxide
    summed_volumes = sum(atom_volume_total)
    
    #Calculate packing fraction. Append values to dictionary with the Materials Project ID as key value
    packing_frac = summed_volumes/oxide_volume
    property_dict_all[str(mp_id)] = pretty_formula,composition,elastic_tensor,packing_frac,density,e_above_hull
    
    #Clear values for next loop
    atom_sample.clear()
    atom_count.clear()
    atom_radii.clear()
    atom_volume.clear()
    atom_volume_total.clear()

742.8107755000007
27.7116966666604
219.57035041666728
1159.8603605555554
71.92039785714321
1329.8974249999994
146.8974075
479.79283499999957
238.16539833333826
284.0176400000001
114.18724312500217
44.34646390625652
0
0
0
0
0
0
30.927984583333235
219.41276299999845
269.5498559999985
205.3349304999985
180.36244649999844
186.64681199999845
151.21475899999837
222.7303359999988
332.5251509999987
234.96431399999818
305.78257249999876
226.12551899999866
213.15853499999804
193.55487750000043
236.15484583333358
205.3391008333332
221.20697999999982
247.14778583333353
153.80105916666676
245.95987958333288
271.82604249999986
288.7631333333336
195.5763741666665
179.73456916666652
203.42069666666697
184.36821416666672
195.19245208333302
255.4725699999998
210.62898000000007
183.529926666667
252.87844999999987
201.66607500000032
258.3297349999998
191.63634666666684
192.08629083333318
163.81981750000028
274.47591416666614
291.9794483333331
269.73427666666663
147.95029666666613
258.6888150000002
238.571

11.746216481480332
22.501968333331845
0
0
0
14.113738854167224
11.790083055554845
0
8.42398361110952
12.423622499999176
11.265508333332619
66.96254861111184
21.635334499999992
0
0
1069.6785519999992
0
147.13967000000673
280.2523715625007
0
117.7889329166666
151.17630725000097
58.22620833333225
0.17758285714997157
202.28703375000023
0
4.723153000006697
0
41.327405000002315
0
6.961826666667115
0
46.607914561966
0
1.6131517708348753
0
504.0450027083283
402.74006791666574
16.2466309722209
0
0
38.86935370833244
355.0663394078946
1.4831636111107827
795.8246807142864
4.388376249998771
38.39453553571737
697.5065724999992
149.5951583333337
360.1960791666663
5.082156666667004
16.375501458332664
1.3641658333334306
11.391343888886851
26.497805624998705
18.039830833332715
8.570851041666216
7.820674166666208
7.699101666664987
70.24511083333351
289.63312611111115
145.9245791666657
21.369046249999357
82.68937666666609
521.0174395833329
11.152319166665592
176.86405666666616
19.227654166666497
296.96958

0
2.500516666666286
20.896565833333547
1.529228125004778
17.70454555555112
1.1933313888885522
62.14407333333316
30.536187249992963
0
169.270385714273
12.99607205127895
100.04932499998631
0
58.963053000000265
0
0
0
0
0
176.21368541666672
24.884984375005192
0
0
99.68930324999903
47.37000999999719
274.38637041666647
0
0.7428426923086207
0
0
0
348.81896699999794
0
0
79.18052541666398
0
25.730518999994345
0
0
0
4.1330100734358055
0
0
1.6022637419341734
138.17533055555754
1417.2017361111102
51.04190862499891
19.86994783333529
151.23966499999318
191.26312172413762
5.010760588207042
133.6879470312482
0
0
105.19952107143294
504.73177477272736
233.60281809523008
2.0710312127372177
0
995.7294393181808
51.504253000000055
19.931692954537894
68.00688615384232
28.936189999997808
663.9902785714327
3.0852059090769046
35.676972499995685
81.45523750000017
86.46262386363368
125.39277775000014
0
49.718245624999824
0
542.5408269444416
0
0
9.646483333334288
0
19.25172666666608
0
0
92.98808124999968
0
0
0
0
0

173.26797566667017
0
1265.7240386250005
1929.5675240625023
0
0
0
0
720.9014903999922
984.9483518181845
306.4787699999938
18.478695357146968
661.421459687503
58.45391944444511
34.37869898374846
272.1146510714299
676.8110557142863
0
0
0
15.419168611108702
18.72221025001153
0.5223222222179302
114.67446739583131
78.57103863636894
100.33587217391293
62.37881309524873
453.47032773809383
66.74282878049542
0
106.49740037499367
25.89205016666707
128.46231000002462
28.673277142860165
0
0
0
0
8.761115952296006
0
56.85569023809833
0
0
0
0
0
0
0
364.0705169270806
0
0
0
0
32.01956964285557
48.922973125002045
86.72934722222081
27.303794285713057
246.6396283333321
496.4036416666655
591.3546658333341
751.1626850000015
0
0
364.9849174107151
423.948338500006
34.14113170454769
38.53602086764685
0
0
244.23748362500052
307.2924002500006
353.0880816071464
174.4796493055514
374.5004215277765
13.86453399999965
561.2689803333337
374.4960731666671
287.1960293154743
327.6872857263078
337.08093056547426
232.607417

0
8.159814999999071
88.48861806818142
111.81692589504965
719.4871771428595
0
72.55905575395971
0
91.97159947916856
51.57186025000726
0
0
0
0
0
0
0
0
1.2690637500005764
0
0
0
0
0
0
0
0
0
0
0
0
0
196.9076775000005
1918.1422016041613
0
0
0
0
25.284671081077903
200.32673749999663
525.5588136805525
260.30675638888744
1487.5052079999964
67.72143688888832
0
0
698.8818337499998
10.144106875005221
0
0
22.65782062499877
0
36.36420037499288
181.91128166667346
0
92.31675999999655
10.113191666667909
0
0
0
20.21541968750018
26.350076785711884
95.20133715985813
0
180.69223498076957
142.99787539062604
142.02639307617292
0
0
0
0
0
0
0
2.781717777774695
0
112.80663749999987
0
0
74.0734481249996
0
1411.9939437500002
3178.606189374996
213.8615420833272
2533.3128243749998
1649.902665595234
2986.477465
2988.4287789583323
4.137791666668278
0
0
0
0
0
135.10735608333223
0
45.21020928570163
77.80818214285645
28.908943571434698
414.1405650000012
0
0
435.29926921875364
0
0
0
0
0
0
0
23.41702962501202
0
359.924472

...

0
479.38920691666544
4.118373600002556
4.2046009000014095
0
27.513505869563026
6.09256769230182
180.73108150112117
36.42841770833449
0
0
6.452811923077562
147.87558215516407
281.1816189620471
0
173.40924507716426
0
314.07671046171436
404.8109177941237
36.882189090905015
44.53717322916617
500.4526232000046
175.3182366847863
450.0120965000001
283.3294689285699
7.140662916666152
223.98230499999806
385.3194567288938
187.660851656692
7.664048571427884
148.71626904099244
110.16707999999475
0.9368978571355768
0
0
0
151.7274165625011
0
324.04252226851946
0
360.40083965278046
0
0
101.82243290038784
25.293026979163358
0
214.04271120000294
414.7127259883705
478.65535074035216
575.4225231875125
35.94914381944658
131.362311623473
5.30532499999925
158.2585462222248
333.66106901587324
115.75493550876992
187.1531675
56.212376875000025
23.624500999998688
74.43347721719019
67.28517385714383
71.45877880953489
156.6520213623237
164.980277437496
30.47599249998978
57.72013128571274
88.03616195834519
59.2397

0
23.738100953568875
0
0
0.4937594907454823
0
10.630598411461634
5.263041111112976
0
1138.3465308888879
0
6.6326110156254
0
2.3676057812496154
0
0
0
7.800350952381407
19.281762058827923
25.270172569433313
26.53199083333302
0
40.7210629411745
0
178.73231183256166
9.010835303025821
0
0
7.867751764705666
9.616044840422333
49.34688944444687
105.51221296874935
15.604186547616195
0
14.170302018516878
0.11448906249977853
21.384840520826565
5.028926329785577
0
0
0
0
0
0
0
0
0
0
81.0367819444453
33.06446975001087
27.138277625001095
0.48199488095157506
8.797445454544306
0
0
0
0
57.787807142856806
25.8633294047641
536.8196030303052
264.7568001041645
1.2347453846235368
0
37.59618883333271
74.20610662500238
0
176.30353611110027
0
0
41.53385739584259
60.69839399999655
6.784066706348568
6.34993925925631
0
3.513483020832986
86.90046704175191
11.327470520833494
5.188468620795916
12.565539705883921
50.258536976749646
257.3123728571476
76.87173093749955
63.055310000001086
26.583773125000043
68.8313656666

0
4.639772307701406
0
16.46601673611059
0
0
0
0
0
145.3753752343756
157.51831812499972
109.0110925976573
67.3393305357255
19.85434379999962
14.892313799999002
0
222.82462302498462
120.4302660515948
186.7098620000096
119.50066731249808
379.321197821425
0
126.08757600794007
3.2833131887768374
120.51470828571276
10.472568312497543
165.1562972222207
0
149.0691670486246
689.0752043478275
49.83157333334454
789.6688475192288
254.57789747222125
109.41936147818154
148.38406484650247
686.2401740036228
118.51538528985284
5.451526900000836
17.071122058823995
0
164.25358420591874
0
0
0
38.72102983333292
0
0
2.8043434143434354
631.0012393055473
376.8129749209521
62.49176338435358
6.45013880679457
6.077504047662607
3.2332922916671336
7.17793542892764
81.60106131636891
546.4360383959947
0
0
0
0
155.737972007576
69.7159413333317
30.609987625000556
13.94417819444893
63.14076380814004
0
406.936191285884
100.44561901667137
37.515188892048634
16.104370938937684
12.093173095240495
0
0
0
51.89370495043111
59

976.8663748611112
0
11.230527134117985
88.76511558064504
54.763416746478555
117.45484291496578
495.8545966920358
111.58762314484783
32.07173965518084
17.13465218390553
30.62762893601345
424.85511690972635
46.678223461550594
212.82864462673334
84.87252235135223
20.373926399706477
173.33932522159046
0
42.6797393237095
9.132114880952003
20.83512581797109
117.12503543981612
461.5051731770832
24.60615750000006
6.56478503471547
382.4113724348912
43.28610171773928
203.00211089743135
41.03811391052048
447.97437630945234
28.517968583332554
36.23980943395111
49.22770606306237
95.99252592548257
146.3634205176918
176.4539703984367
491.4949657812446
334.9070354374968
46.06435792508368
0.6164950000018266
342.8362497211754
253.618379717631
72.7775537361115
0
147.8411242466997
219.0366594999995
257.2573674893146
286.21395828571394
277.83590000000123
0
0
133.31997387523575
0
0
267.7768549523822
327.9621329742115
616.3954107031237
1075.1833653
0
267.9148224437373
385.51260187136995
11.222962718748342
11

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

### 2.2 Convert our dictionary to a more pleasant looking dataframe. Pull the values from each key, separate them, and shove everything into a dataframe.

In [267]:
#Create blank lists for appending values
mp_id = []
pretty_formula = []
composition = []
ionic_packing_frac = []
density = []
elastic_tensor = []
e_above_hull = []
#Loop through all keys and extract their values. Append values into individual list arrays
for key in property_dict_all.keys():
    mp_id.append(key)
    values = property_dict_all.get(key)
    pretty_formula.append(values[0])
    composition.append(values[1])
    elastic_tensor.append(values[2])
    ionic_packing_frac.append(values[3])
    density.append(values[4])
    e_above_hull.append(values[5])    
#Stack arrays into a Numpy array. From here will append into a dataframe. Display dataframe below.
df = np.column_stack((mp_id,pretty_formula,composition,ionic_packing_frac,density,elastic_tensor,e_above_hull))
df_oxide = pd.DataFrame(df,columns=["mp_id","Formula","Composition","IPF","Density","Elastic_Tensor","E_above_Hull"])
display(df_oxide)

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull
0,mp-1042,HfSi,"{'Hf': 1.0, 'Si': 1.0}",0.0940669,9.98367,"{'G_Reuss': 92.0, 'G_VRH': 96.0, 'G_Voigt': 99...",0
1,mp-10651,YbSi,"{'Yb': 1.0, 'Si': 1.0}",0.128123,7.13753,"{'G_Reuss': 37.0, 'G_VRH': 38.0, 'G_Voigt': 39...",0
2,mp-1066975,TmSi,"{'Tm': 1.0, 'Si': 1.0}",0.148546,7.89033,None,0
3,mp-1067235,BaSi,"{'Ba': 1.0, 'Si': 1.0}",0.226513,4.28615,None,0
4,mp-10711,VSi2,"{'V': 1.0, 'Si': 2.0}",0.0857882,4.64084,"{'G_Reuss': 146.0, 'G_VRH': 147.0, 'G_Voigt': ...",0
...,...,...,...,...,...,...,...
2510,mp-1196506,Zr5Si6H36C12(Br10O3)2,"{'Zr': 5.0, 'Si': 6.0, 'H': 36.0, 'C': 12.0, '...",0.0737496,2.41958,None,0
2511,mp-1200353,Cs4K3U3Si4O20F,"{'Cs': 4.0, 'K': 3.0, 'U': 3.0, 'Si': 4.0, 'O'...",0.509664,4.6898,None,0
2512,mp-1210416,Na4CaZr2TiSi4(O8F)2,"{'Na': 4.0, 'Ca': 1.0, 'Zr': 2.0, 'Ti': 1.0, '...",0.478534,3.395,None,0
2513,mp-1210579,Na2CaZrSi2O7F2,"{'Na': 2.0, 'Ca': 1.0, 'Zr': 1.0, 'Si': 2.0, '...",0.426514,3.20041,None,0


In [268]:
df_oxide.dtypes

mp_id             object
Formula           object
Composition       object
IPF               object
Density           object
Elastic_Tensor    object
E_above_Hull      object
dtype: object

In [269]:
df_oxide['Elastic_Tensor']

0       {'G_Reuss': 92.0, 'G_VRH': 96.0, 'G_Voigt': 99...
1       {'G_Reuss': 37.0, 'G_VRH': 38.0, 'G_Voigt': 39...
2                                                    None
3                                                    None
4       {'G_Reuss': 146.0, 'G_VRH': 147.0, 'G_Voigt': ...
                              ...                        
2510                                                 None
2511                                                 None
2512                                                 None
2513                                                 None
2514                                                 None
Name: Elastic_Tensor, Length: 2515, dtype: object

In [270]:
df_oxide=df_oxide.dropna()

In [271]:
df_oxide

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull
0,mp-1042,HfSi,"{'Hf': 1.0, 'Si': 1.0}",0.0940669,9.98367,"{'G_Reuss': 92.0, 'G_VRH': 96.0, 'G_Voigt': 99...",0
1,mp-10651,YbSi,"{'Yb': 1.0, 'Si': 1.0}",0.128123,7.13753,"{'G_Reuss': 37.0, 'G_VRH': 38.0, 'G_Voigt': 39...",0
4,mp-10711,VSi2,"{'V': 1.0, 'Si': 2.0}",0.0857882,4.64084,"{'G_Reuss': 146.0, 'G_VRH': 147.0, 'G_Voigt': ...",0
13,mp-1118,SiNi2,"{'Si': 1.0, 'Ni': 2.0}",0.123319,7.34741,"{'G_Reuss': 62.0, 'G_VRH': 64.0, 'G_Voigt': 65...",0
14,mp-11190,VSi2,"{'V': 1.0, 'Si': 2.0}",0.0857735,4.64004,"{'G_Reuss': 146.0, 'G_VRH': 147.0, 'G_Voigt': ...",0.00316222
...,...,...,...,...,...,...,...
2284,mp-6624,Tb2Fe2Si2C,"{'Tb': 2.0, 'Fe': 2.0, 'Si': 2.0, 'C': 1.0}",0.133449,7.58353,"{'G_Reuss': 76.0, 'G_VRH': 81.0, 'G_Voigt': 85...",0
2299,mp-1025316,Ho2Fe2Si2C,"{'Ho': 2.0, 'Fe': 2.0, 'Si': 2.0, 'C': 1.0}",0.150549,7.92792,"{'G_Reuss': 80.0, 'G_VRH': 84.0, 'G_Voigt': 87...",0
2300,mp-1025357,Lu2Fe2Si2C,"{'Lu': 2.0, 'Fe': 2.0, 'Si': 2.0, 'C': 1.0}",0.145499,8.51918,"{'G_Reuss': 82.0, 'G_VRH': 85.0, 'G_Voigt': 88...",0
2388,mp-570102,HoSiRu2C,"{'Ho': 1.0, 'Si': 1.0, 'Ru': 2.0, 'C': 1.0}",0.104817,8.95076,"{'G_Reuss': 75.0, 'G_VRH': 77.0, 'G_Voigt': 79...",0


### 2.3 Observe the reduction from all oxide species to the limited set for stable structures. Plot the data below.

In [272]:
print('Number of structures available on the MP database: %s' % len(data))
print('Number of structures below %.1f meV: %s' %(energy_cutoff_value, len(df_oxide)))

Number of structures available on the MP database: 8932
Number of structures below 1.0 meV: 528


In [273]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. Commented out pieces are height and width. 
#For plot in browser I prefer long horizontal plots. For presentations square images may be preferred. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_IPF = go.Layout(title= "Oxide Query IPF vs. Density", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Density \: [g/cm^{3}]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Ionic \: Packing \: Fraction \: [IPF]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= True                
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column.
trace_all = go.Scatter(x = df_oxide.Density, y = df_oxide.IPF, mode = 'markers',
                    marker=dict(size=12, color='black'), text = df_oxide.Formula, name = 'All Queries')

trace_Al2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Al2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'Al2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='green'), text = 'Al2O3', name = 'Al2O3')
trace_Cr2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Cr2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'Cr2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='blue'), text = 'Cr2O3', name = 'Cr2O3')
trace_Y2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Y2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'Y2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='orange'), text = 'Y2O3', name = 'Y2O3')
trace_SiO2 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'SiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'SiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='magenta'), text = 'SiO2', name = 'SiO2')

data_IPF = [trace_all,trace_Al2O3,trace_Cr2O3,trace_Y2O3,trace_SiO2]#,trace0,trace2]
fig_IPF = go.Figure(data_IPF, layout=layout_IPF)
iplot(fig_IPF)

## 3. Filtering data for elasticity

### 3.1 Observe the reduction from all oxide species to the limited set for stable structures, and ones with mechanical property data


In [274]:
print('Number of structures available on the MP database: %s' % len(data))
print('Number of structures below %.1f meV and with elasticity data: %s' %(energy_cutoff_value, len(df_oxide)))

Number of structures available on the MP database: 8932
Number of structures below 1.0 meV and with elasticity data: 528


### 3.2 Pull dataframe column with dictionary of elastic constants. Create separate columns for each value instead of inset in dictionary.

In [275]:
#Pull dataframe column with dictionary of elastic constants
elastics = df_oxide.Elastic_Tensor
G_VRH = []
K_VRH = []
Elastic_Anisotropy = []
poisson_ratio = []
Y_Modulus = []

for k in elastics.index: #len(df_oxide))
    G_VRH.append(elastics[k].get('G_VRH'))
    K_VRH.append(elastics[k].get('K_VRH'))

    Elastic_Anisotropy.append(elastics[k].get('elastic_anisotropy'))
    poisson_ratio.append(elastics[k].get('poisson_ratio'))
    Y_Modulus.append(2*elastics[k].get('G_VRH')*(1+elastics[k].get('poisson_ratio')))
    
elasticity_data = np.column_stack((G_VRH, K_VRH, Elastic_Anisotropy, poisson_ratio, Y_Modulus))

df_elasticity = pd.DataFrame(elasticity_data,columns=["G_VRH", 
                                                      "K_VRH", 
                                                      "Elastic_Anisotropy", 
                                                      "poisson_ratio",
                                                     "Y_Modulus"])



df_oxide = df_oxide.reset_index(drop=True)
df_elasticity = df_elasticity.reset_index(drop=True)
df_oxide = pd.concat([df_oxide,df_elasticity],axis=1)
display(df_oxide)

#Remove erroneous poisson ratios
void_test = []
row_delete = []
for k in range(0,len(df_elasticity)):
    if df_oxide['poisson_ratio'].iloc[k] < 0:
        row_delete.append(k)
        

index = df_oxide.index[row_delete]
df_oxide.drop(index,axis=0,inplace=True)

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull,G_VRH,K_VRH,Elastic_Anisotropy,poisson_ratio,Y_Modulus
0,mp-1042,HfSi,"{'Hf': 1.0, 'Si': 1.0}",0.0940669,9.98367,"{'G_Reuss': 92.0, 'G_VRH': 96.0, 'G_Voigt': 99...",0,96.0,153.0,0.42,0.24,238.08
1,mp-10651,YbSi,"{'Yb': 1.0, 'Si': 1.0}",0.128123,7.13753,"{'G_Reuss': 37.0, 'G_VRH': 38.0, 'G_Voigt': 39...",0,38.0,57.0,0.27,0.22,92.72
2,mp-10711,VSi2,"{'V': 1.0, 'Si': 2.0}",0.0857882,4.64084,"{'G_Reuss': 146.0, 'G_VRH': 147.0, 'G_Voigt': ...",0,147.0,174.0,0.05,0.17,343.98
3,mp-1118,SiNi2,"{'Si': 1.0, 'Ni': 2.0}",0.123319,7.34741,"{'G_Reuss': 62.0, 'G_VRH': 64.0, 'G_Voigt': 65...",0,64.0,193.0,0.25,0.35,172.80
4,mp-11190,VSi2,"{'V': 1.0, 'Si': 2.0}",0.0857735,4.64004,"{'G_Reuss': 146.0, 'G_VRH': 147.0, 'G_Voigt': ...",0.00316222,147.0,175.0,0.05,0.17,343.98
...,...,...,...,...,...,...,...,...,...,...,...,...
523,mp-6624,Tb2Fe2Si2C,"{'Tb': 2.0, 'Fe': 2.0, 'Si': 2.0, 'C': 1.0}",0.133449,7.58353,"{'G_Reuss': 76.0, 'G_VRH': 81.0, 'G_Voigt': 85...",0,81.0,126.0,0.61,0.24,200.88
524,mp-1025316,Ho2Fe2Si2C,"{'Ho': 2.0, 'Fe': 2.0, 'Si': 2.0, 'C': 1.0}",0.150549,7.92792,"{'G_Reuss': 80.0, 'G_VRH': 84.0, 'G_Voigt': 87...",0,84.0,129.0,0.51,0.23,206.64
525,mp-1025357,Lu2Fe2Si2C,"{'Lu': 2.0, 'Fe': 2.0, 'Si': 2.0, 'C': 1.0}",0.145499,8.51918,"{'G_Reuss': 82.0, 'G_VRH': 85.0, 'G_Voigt': 88...",0,85.0,141.0,0.37,0.25,212.50
526,mp-570102,HoSiRu2C,"{'Ho': 1.0, 'Si': 1.0, 'Ru': 2.0, 'C': 1.0}",0.104817,8.95076,"{'G_Reuss': 75.0, 'G_VRH': 77.0, 'G_Voigt': 79...",0,77.0,176.0,0.30,0.31,201.74


In [254]:
# export all the dataframes for all seven datasets.
df_oxide.to_csv (r'C:\Users\Kayla Yano\Class\Trial\Cr_coumpounds_dataframe.csv', index = True, header=True)


### 3.3.1 Plot material properties from dataframe

In [276]:
#Layout design. Can change title, font, x/y-axis etc. Commented out pieces are height and width. For plot in browser I prefer long horizontal plots. For presentations square images may be preferred. 
#Image can be directly saved by hovering over image and clicking camera icon. 
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

layout_IPF = go.Layout(title= "Ti_based Query IPF vs. Density", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Density \: [g/cm^{3}]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Ionic Packing Fraction \: [IPF]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= True                
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_all = go.Scatter(x = df_oxide.Density, y = df_oxide.IPF, mode = 'markers',
                    marker=dict(size=12, color='black'), text = df_oxide.Formula, name = 'All Queries')

trace_TiC = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiC') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'TiC') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='green'), text = 'TiC', name = 'TiC')
trace_TiN = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiN') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'TiN') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='blue'), text = 'TiN', name = 'TiN')
trace_TiB = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiB') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'TiB') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='orange'), text = 'TiB', name = 'TiB')
trace_TiO2 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'TiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='magenta'), text = 'TiO2', name = 'TiO2')

data_IPF = [trace_all,trace_TiC,trace_TiN,trace_TiB,trace_TiO2]
fig_IPF = go.Figure(data_IPF, layout=layout_IPF)
iplot(fig_IPF)

### 3.3.2 Plot material properties from dataframe using 3rd property as color key

In [277]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
layout_IPF_shear_modulus = go.Layout(title= "C-based Compounds Bulk Modulus vs. Shear Modulus", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Shear \: Modulus \: [GPa]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Bulk \: Modulus [GPa]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= False              
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_all_shear = go.Scatter(x = abs(df_oxide.G_VRH), y = df_oxide.K_VRH, mode = 'markers', text = df_oxide.Formula,
                            marker=dict(size=12, color=(df_oxide['poisson_ratio']),
                                colorbar = dict(title={ 'text': "Poisson Ratio", 
                                                     'font': {'family':'Georgia', 'size': 18}} , 
                                              tickfont={'family':'Georgia', 'size': 16 })))#, name = 'All Queries'))

                             
data_IPF_shear = [trace_all_shear]
fig_IPF_shear = go.Figure(data_IPF_shear, layout=layout_IPF_shear_modulus)
iplot(fig_IPF_shear)

In [278]:
layout_test = go.Layout(title= "Si compounds", hovermode= 'closest',
                  font = dict(family='Times New Roman',size=36),
                  xaxis= dict(title= 'Density [g/cm<sup>-3</sup>]',zeroline= False, gridwidth= 2), #To change range put command inside xaxis dictionary after gridwidth range=[-XX,XX]
                  yaxis= dict(title= "Young's Modulus [GPa]",zeroline= False, gridwidth= 2,range=[0,600]), #To change range put command inside yaxis dictionary after gridwidth range=[-YY,YY]
                  height = 800,
                  width = 800,
    showlegend= False              
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_test = go.Scatter(x = df_oxide.Density, y = df_oxide.Y_Modulus, mode = 'markers', text = df_oxide.Formula,
                           marker=dict(size=12, color=(df_oxide['poisson_ratio']),
                               colorbar = dict(title={ 'text': "Poisson Ratio", 
                                                    'font': {'family':'Georgia', 'size': 18}},  
                                             tickfont={'family':'Georgia', 'size': 16 })), )#, name = 'All Queries'))


import plotly.express as px
data_test = [trace_test]
fig_test = go.Figure(data_test, layout=layout_test)
# fig_test.update_layout(color_continuous_scale=px.colors.sequential.Viridis)
iplot(fig_test)